## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Clinical De-identification for Arabic](https://aws.amazon.com/marketplace/pp/prodview-fslxwvs65fwuu).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Arabic

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: [ar.deid.clinical](https://nlp.johnsnowlabs.com/2023/06/22/clinical_deidentification_ar.html)
- **Model Description**: This pipeline can be used to deidentify Arabic PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate CONTACT, NAME, DATE, ID, LOCATION, AGE, PATIENT, HOSPITAL, ORGANIZATION, CITY, STREET, USERNAME, SEX, IDNUM, EMAIL, ZIP, MEDICALRECORD, PROFESSION, PHONE, COUNTRY, DOCTOR, SSN, ACCOUNT, LICENSE, DLN and VIN

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "ar-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    - **masked**: Default policy that masks entities with their type.

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

    - **masked_with_chars**: Masks entities with asterisks (\*).
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked"
}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = ["""ملاحظات سريرية - مريض السكري:
التاريخ: 30 مايو 2023
اسم المريض: فاطمة علي
العنوان: شارع الحرية، مبنى رقم 456، حي السلام، القاهرة
الرمز البريدي: 67890
البلد: مصر
اسم المستشفى: مستشفى الشفاء
اسم الطبيب: د. محمد صلاح
تفاصيل الحالة:
المريضة فاطمة علي، البالغة من العمر 42 عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.
الخطة:
تم وصف دواء فموي لخفض مستوى السكر في الدم.
يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.
يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.
يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.
تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.""",

"""ملاحظات سريرية - مريض الربو:
التاريخ: 30 مايو 2023
اسم المريضة: ليلى حسن
العنوان: شارع المعرفة، مبنى رقم 789، حي الأمانة، جدة
الرمز البريدي: 54321
البلد: المملكة العربية السعودية
اسم المستشفى: مستشفى النور
اسم الطبيب: د. أميرة أحمد
تفاصيل الحالة:
المريضة ليلى حسن، البالغة من العمر 35 عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة.
الخطة:
تم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.
يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة.
يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة.
يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو.
تعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح""",

"""ملاحظات سريرية - مريض الضغط العالي:
التاريخ: 30 مايو 2023
اسم المريض: خالد محمد
العنوان: شارع الشفاء، مبنى رقم 321، حي السعادة، الإسكندرية
الرمز البريدي: 12345
البلد: مصر
اسم المستشفى: مستشفى الرحمة
اسم الطبيب: د. ليلى علي
تفاصيل الحالة:
المريض خالد محمد، البالغ من العمر 60 عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة.
الخطة:
تم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام.
يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر.
تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام.
يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين.
ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.""",

"""ملاحظات سريرية - مريضة الغدة الدرقية:
التاريخ: 30 مايو 2023
اسم المريضة: سارة عبد الرحمن
العنوان: شارع النجاح، مبنى رقم 987، حي الأمل، القاهرة
الرمز البريدي: 67890
البلد: مصر
اسم المستشفى: مستشفى الأمل
اسم الطبيب: د. أحمد مصطفى
تفاصيل الحالة:
المريضة سارة عبد الرحمن، البالغة من العمر 45 عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية.
الخطة:
تم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام.
يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر.
تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام.
ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج.
تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.""",
]


sample_text = """ملاحظات سريرية - مريض الربو:
التاريخ: 30 مايو 2023
اسم المريض: أحمد سليمان
العنوان: شارع السلام، مبنى رقم 555، حي الصفاء، الرياض
الرمز البريدي: 54321
البلد: المملكة العربية السعودية
اسم المستشفى: مستشفى الأمانة
اسم الطبيب: د. ريم الحمد
تفاصيل الحالة:
المريض أحمد سليمان، البالغ من العمر 30 عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة.
الخطة:
تم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.
يجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام.
ينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين.
يجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة.
تعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح."""

### JSON

#### Example 1: masked (default-policy)

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

ملاحظات سريرية - مريض الربو:
التاريخ: [تاريخ] [تاريخ]
اسم المريض: [دكتور]
العنوان: شارع السلام، مبنى رقم [الرمز البريدي] حي [الموقع]
الرمز البريدي: [الرمز البريدي]
البلد: [المدينة] [البلد]
اسم المستشفى: مستشفى الأمانة
اسم الطبيب: د. [دكتور]
تفاصيل الحالة:
المريض أحمد سليمان، البالغ من العمر [العمر]عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة.
الخطة:
تم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.
يجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام.
ينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين.
يجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة.
تعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح.


#### Example 2: obfuscated

In [11]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
print(response_json["predictions"][0])

ملاحظات سريرية - مريض الربو:
التاريخ: 30 يونيو 2024
اسم المريض: جاد قفطان
العنوان: شارع السلام، مبنى رقم 222، حي شارع المستقبل
الرمز البريدي: 29630
البلد: زغوان الغربية أوزبكستان
اسم المستشفى: مستشفى الأمانة
اسم الطبيب: د. رينا سرحان
تفاصيل الحالة:
المريض أحمد سليمان، البالغ من العمر 20عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة.
الخطة:
تم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.
يجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام.
ينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين.
يجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة.
تعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح.


### JSON Lines

In [13]:
def create_jsonl(records, masking_policy=None):
    json_records = []

    if isinstance(records, str):
        records = [records]

    for text in records:
        record = {"text": text}

        if masking_policy is not None:
            record["masking_policy"] = masking_policy
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record, ensure_ascii=False) for record in json_records)
    return json_lines


#### Example 1: masked (default-policy)

In [14]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="masked")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ]\nاسم المريض: [دكتور]\nالعنوان: شارع السلام، مبنى رقم [الرمز البريدي] حي [الموقع]\nالرمز البريدي: [الرمز البريدي]\nالبلد: [المدينة] [البلد]\nاسم المستشفى: مستشفى الأمانة\nاسم الطبيب: د. [دكتور]\nتفاصيل الحالة:\nالمريض أحمد سليمان، البالغ من العمر [العمر]عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة.\nالخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.\nيجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام.\nينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين.\nيجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة.\nتعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح."}


#### Example 2: obfuscated

In [16]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="obfuscated")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "ملاحظات سريرية - مريض الربو:\nالتاريخ: 30 يونيو 2024\nاسم المريض: جاد قفطان\nالعنوان: شارع السلام، مبنى رقم 222، حي شارع المستقبل\nالرمز البريدي: 29630\nالبلد: زغوان الغربية أوزبكستان\nاسم المستشفى: مستشفى الأمانة\nاسم الطبيب: د. رينا سرحان\nتفاصيل الحالة:\nالمريض أحمد سليمان، البالغ من العمر 20عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة.\nالخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.\nيجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام.\nينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين.\nيجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة.\nتعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح."}


### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [19]:
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [20]:
# Create JSON and JSON Lines data
input_json_data = {
    "input1.json": json.dumps({"text": docs, "masking_policy": "masked"}, ensure_ascii=False),
    "input2.json": json.dumps({"text": docs, "masking_policy": "obfuscated"}, ensure_ascii=False),
    "input3.json": json.dumps({"text": docs, "masking_policy": "masked_fixed_length_chars"}, ensure_ascii=False),
    "input4.json": json.dumps({"text": docs, "masking_policy": "masked_with_chars"}, ensure_ascii=False),
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}

# Upload JSON and JSON Lines data to S3
for file_name, data in input_json_data.items():
    upload_to_s3(data, file_name)

for file_name, data in input_jsonl_data.items():
    upload_to_s3(data, file_name)


### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [22]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [23]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_json_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------


{'predictions': ['ملاحظات سريرية - مريض السكري:\nالتاريخ: [تاريخ] [تاريخ]\nاسم المريض: [الاسم]\nالعنوان: شارع الحرية، مبنى رقم [الرمز البريدي] حي السلام، [المدينة]\nالرمز البريدي: [الرمز البريدي]\nالبلد: [البلد]\nاسم المستشفى: مستشفى الشفاء\nاسم الطبيب: د. [دكتور]\nتفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر [العمر]عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.\nالخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم.\nيجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.\nيتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.\nيجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.\nتعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.',
  'ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ]\nاسم المريضة: [المريض]\nالعنوان: شارع المعرفة، مبنى رقم [الرمز البري



-------------------------------------------------- obfuscated --------------------------------------------------


{'predictions': ['ملاحظات سريرية - مريض السكري:\nالتاريخ: 30 يونيو 2024\nاسم المريض: عبدالغني جمالة\nالعنوان: شارع الحرية، مبنى رقم 925، حي السلام، القصر\nالرمز البريدي: 58147\nالبلد: أوكرانيا\nاسم المستشفى: مستشفى الشفاء\nاسم الطبيب: د. حمدي نصراوي\nتفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر 53عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.\nالخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم.\nيجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.\nيتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.\nيجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.\nتعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.',
  'ملاحظات سريرية - مريض الربو:\nالتاريخ: 30 يونيو 2024\nاسم المريضة: نوح شقيري\nالعنوان: شارع المعرفة، مبنى رقم 814، حي شارع الحيوانات الأليفة\nا



-------------------------------------------------- masked_fixed_length_chars --------------------------------------------------


{'predictions': ['ملاحظات سريرية - مريض السكري:\nالتاريخ: ٭٭٭٭ ٭٭٭٭\nاسم المريض: ٭٭٭٭\nالعنوان: شارع الحرية، مبنى رقم ٭٭٭٭ حي السلام، ٭٭٭٭\nالرمز البريدي: ٭٭٭٭\nالبلد: ٭٭٭٭\nاسم المستشفى: مستشفى الشفاء\nاسم الطبيب: د. ٭٭٭٭\nتفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر ٭٭٭٭عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.\nالخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم.\nيجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.\nيتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.\nيجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.\nتعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.',
  'ملاحظات سريرية - مريض الربو:\nالتاريخ: ٭٭٭٭ ٭٭٭٭\nاسم المريضة: ٭٭٭٭\nالعنوان: شارع المعرفة، مبنى رقم ٭٭٭٭ حي ٭٭٭٭\nالرمز البريدي: ٭٭٭٭\nالبلد: ٭٭٭٭ ٭٭٭٭\nاسم المستشفى: 



-------------------------------------------------- masked_with_chars --------------------------------------------------


{'predictions': ['ملاحظات سريرية - مريض السكري:\nالتاريخ: [٭٭٭٭٭] [٭٭]\nاسم المريض: [٭٭٭٭٭٭٭]\nالعنوان: شارع الحرية، مبنى رقم [٭٭] حي السلام، [٭٭٭٭٭]\nالرمز البريدي: [٭٭٭]\nالبلد: [٭]\nاسم المستشفى: مستشفى الشفاء\nاسم الطبيب: د. [٭٭٭٭٭٭٭]\nتفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر ٭٭عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.\nالخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم.\nيجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.\nيتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.\nيجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.\nتعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.',
  'ملاحظات سريرية - مريض الربو:\nالتاريخ: [٭٭٭٭٭] [٭٭]\nاسم المريضة: [٭٭٭٭٭٭]\nالعنوان: شارع المعرفة، مبنى رقم [٭٭] حي [٭٭٭٭٭٭٭٭٭٭]\nالرمز البريدي: [٭٭٭]\nال

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [25]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [26]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print("-"*50, policy_name ,"-"*50)
    retrieve_jsonlines_output_from_s3(validation_file_name)
    print("\n")

-------------------------------------------------- masked --------------------------------------------------
{"predictions": "ملاحظات سريرية - مريض السكري:\nالتاريخ: [تاريخ] [تاريخ]\nاسم المريض: [الاسم]\nالعنوان: شارع الحرية، مبنى رقم [الرمز البريدي] حي السلام، [المدينة]\nالرمز البريدي: [الرمز البريدي]\nالبلد: [البلد]\nاسم المستشفى: مستشفى الشفاء\nاسم الطبيب: د. [دكتور]\nتفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر [العمر]عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.\nالخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم.\nيجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.\nيتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.\nيجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.\nتعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها."}
{"predictions": "م

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

